In [1]:
import matplotlib.pyplot as plt
import wntr
import wntr.network.controls as controls
import sqlite3
import csv
import time
import pandas as pd

In [2]:
# Create the network
inp_file = '../EPANET_topologies/epanetWADI_Measuredv3 - Run55.inp'

wn = wntr.network.WaterNetworkModel(inp_file)

# Set option for step-by-step simulation
wn.options.time.duration = 5

list_header = []
node_list = list(wn.node_name_list)
link_list = list(wn.link_name_list)

In [3]:
def get_node_list_by_type(list, type):
    result = []
    for node in list:
        if wn.get_node(node).node_type == type:
            result.append(str(node))
    return result

def get_link_list_by_type(list, type):
    result = []
    for link in list:
        if wn.get_link(link).link_type == type:
            result.append(str(link))
    return result

def create_node_header(list):
    result = []
    for node in list:
        result.append(node+"_LEVEL")
    return result

def create_link_header(list):
    result = []
    for link in list:
        result.append(link+"_FLOW")
        result.append(link+"_STATUS")
    return result

def get_controls(list):
    result = []
    for control in list:
        result.append(wn.get_control(control))
    return result

def create_control_dict(actuator):
    dummy_condition = controls.ValueCondition(wn.get_node('T1'), 'level', '>=', -1)
    act_dict = dict.fromkeys(['actuator', 'parameter', 'value', 'condition', 'name'])
    act_dict['actuator'] = wn.get_link(actuator)
    act_dict['parameter'] = 'status'
    act_dict['condition'] = dummy_condition
    act_dict['name'] = actuator + "_control"
    if type(wn.get_link(actuator).status) is int:
                act_dict['value'] = act_dict['actuator'].status
    else:
                act_dict['value'] = act_dict['actuator'].status.value   
    return act_dict
    
def register_results(results):
    values_list = []    
    values_list.extend([results.timestamp])
    
    #Results are divided into: nodes: reservoir and tanks, links: flows and status
    # Get tanks levels
    for tank in tank_list:
            values_list.extend([wn.get_node(tank).level])

    # Get junction  levels         
    #for junction in junction_list:
    #values_list.extend([wn.get_node(junction).head - wn.get_node(junction).elevation])

    # Get pumps flows and status
    for pump in pump_list:
            values_list.extend([wn.get_link(pump).flow])

            if type(wn.get_link(pump).status) is int:
                values_list.extend([wn.get_link(pump).status])  # The value is an integer
            else:
                values_list.extend([wn.get_link(pump).status.value]) # The value is an attribute of status, integer

    # Get valves flows and status
    for valve in valve_list:
            values_list.extend([wn.get_link(valve).flow])

            if type(wn.get_link(valve).status) is int:
                values_list.extend([wn.get_link(valve).status])
            else:
                values_list.extend([wn.get_link(valve).status.value]) 
    return values_list

In [4]:
dummy_condition = controls.ValueCondition(wn.get_node('T1'), 'level', '>=', -1)

reservoir_list = get_node_list_by_type(node_list, 'Reservoir')
tank_list      = get_node_list_by_type(node_list, 'Tank')
junction_list  = get_node_list_by_type(node_list, 'Junction')
pump_list      = get_link_list_by_type(link_list, 'Pump')
valve_list     = get_link_list_by_type(link_list, 'Valve')

list_header = ["Timestamps"]
aux = create_node_header(tank_list)
list_header.extend(aux)

aux = create_node_header(junction_list)
list_header.extend(aux)

aux = create_link_header(pump_list)
list_header.extend(aux)

aux = create_link_header(valve_list)
list_header.extend(aux)

results_list = []
results_list.append(list_header)

In [5]:
# START STEP BY STEP SIMULATION
sim = wntr.sim.EpanetSimulator(wn)
iteration = 0 
iteration_limit = (14*60)/5
while iteration <= iteration_limit:
    
    #update_actuator_values()
    results = sim.run_sim()
    values_list = register_results(results)
    
    results_list.append(values_list)    
    iteration += 1

In [6]:
with open('ctown_no_minicps.csv', 'w', newline='\n') as f:
    writer = csv.writer(f)
    writer.writerows(results_list)